# MS Prediction Data Cleaning

In [1]:
import pandas as pd

import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

from dateutil import parser
import datetime

from sklearn.model_selection import train_test_split


## Diagnoses labels

Found in `seantis/diagnoses.csv` in the `disease` column.

In [2]:
labels = pd.read_csv(paths.DATA_PATH_SEANTIS/'diagnoses.csv')

In [3]:
labels.head()

,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
0,2011-10,confirmed,Progressive multifokale Leukencephalopathie (PML),2011-10,A0684D32-19C0-4538-AFDE-52959CCD8B63
1,2002-08,confirmed,relapsing_remitting_multiple_sclerosis,2002-06,A0684D32-19C0-4538-AFDE-52959CCD8B63
2,2011-11,confirmed,St.n. symptomatischer Epilepsie mit einfach fo...,2011-11,A0684D32-19C0-4538-AFDE-52959CCD8B63
3,2008-10,confirmed,relapsing_remitting_multiple_sclerosis,2003,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,2000,NaN,relapsing_remitting_multiple_sclerosis,2000,EF6AFC6C-BEB1-43B9-A5F4-79F321FF100C


In [4]:
print("Label distribution:")
print(labels['disease'].value_counts())
print("None: ", labels['disease'].isna().sum(), "\n")
print("Unique patient IDs: ", len(labels['research_id'].unique()))
print("Duplicate patient IDs: ", len(labels['research_id']) - len(labels['research_id'].unique()))

Label distribution:
disease
relapsing_remitting_multiple_sclerosis                                              284
clinically_isolated_syndrome                                                         45
secondary_progressive_multiple_sclerosis                                             27
primary_progressive_multiple_sclerosis                                               18
Schubförmige Multiple Sklerose                                                        3
Multiple Sklerose a.e. primär progredient                                             2
Progressive multifokale Leukencephalopathie (PML)                                     1
St.n. symptomatischer Epilepsie mit einfach fokalen und komplex fokalen Anfällen      1
Schubförmig remittierende Multiple Sklerose (RRMS)                                    1
Multiple Sklerose mit am ehesten progredientem Verlauf                                1
Schubförmig remittierende Multiple Sklerose                                           1
Mult

In [5]:
print("Diagnosis reliability:")
print(labels['diagnosis_reliability'].value_counts())

Diagnosis reliability:
diagnosis_reliability
confirmed      267
status_post     51
suspected       12
Name: count, dtype: int64


In [6]:
# Print missing dates
print("Missing dates:")
print(labels["diagnosis_date"].isna().sum())
print("\n")

# Date column formats
print("Date column formats:")
print(labels["diagnosis_date"].apply(lambda x: len(x)).value_counts())
print("\n")

# Print a 10 character date
print("Example date:")
print(labels["diagnosis_date"][labels["diagnosis_date"].apply(lambda x: len(x)) == 10].iloc[1])

Missing dates:
0


Date column formats:
diagnosis_date
7     270
4      67
10     50
Name: count, dtype: int64


Example date:
2020-10-30


In [7]:
# Guessing the date format from the length and example. Seems to follow english format with YYYY-MM-DD.
date_map = {
    7: "YYYY-MM",
    10: "YYYY-MM-DD",
    4: "YYYY",
}
print(labels["diagnosis_date"].apply(lambda x: len(x))
      .value_counts()
      .reset_index()
      .replace({"diagnosis_date": date_map})
      .rename(columns={"diagnosis_date": "format"}))

       format  count
0     YYYY-MM    270
1        YYYY     67
2  YYYY-MM-DD     50


In [8]:
# Print the entries for the patients with multiple labels
print("Examples Patients with multiple labels:")
display((labels[labels["research_id"].duplicated(keep=False)].sort_values(by=["research_id", "disease_onset_date"])))

Examples Patients with multiple labels:


,diagnosis_date,diagnosis_reliability,disease,disease_onset_date,research_id
114,2013-11,confirmed,relapsing_remitting_multiple_sclerosis,2012-12,05157228-A1C5-4335-B990-AF72B312FF1A
115,2013-04,status_post,clinically_isolated_syndrome,2012-12,05157228-A1C5-4335-B990-AF72B312FF1A
163,2015-02,confirmed,relapsing_remitting_multiple_sclerosis,2014-09,08F1FA27-6213-47AF-B967-DC44CCCE9613
164,2014-10,status_post,clinically_isolated_syndrome,2014-09-28,08F1FA27-6213-47AF-B967-DC44CCCE9613
63,2009-11,confirmed,relapsing_remitting_multiple_sclerosis,2009-01,146E4473-897F-402D-9F33-7943658FB47D
...,...,...,...,...,...
304,1997-10,confirmed,secondary_progressive_multiple_sclerosis,1992,F4D0C12B-8BEC-4B0E-BB80-44909D9A8507
188,2015-07,status_post,clinically_isolated_syndrome,2015-06,F5C813F0-1281-4C7E-87E1-78BA8AC3738A
189,2016-01,confirmed,relapsing_remitting_multiple_sclerosis,2015-06,F5C813F0-1281-4C7E-87E1-78BA8AC3738A
176,2017-03,confirmed,relapsing_remitting_multiple_sclerosis,2015-04,F713ABCC-6E83-45E5-B660-E68CA43024CF


### Problems

- Mix of german and english labels.
- Irrelevant labels
- Class imbalance
- There could be multiple diagnoses per patient
- The date column contains multiple formats, YYYY-MM, YYYY, YYYY-MM-DD
- Some diagnoses are not confirmed, could be detrimental to model training. In old approach they excluded these.

### Conclusion

- Map all non-english labels to their corresponding english class
- Exclude irrelevant labels (like clinically_isolated_syndrome). Keep relevant labels (relapsing_remitting_multiple_sclerosis (RRMS), secondary_progressive_multiple_sclerosis (SPMS), primary_progressive_multiple_sclerosis (PPMS))
- Do dataset-splits stratified
- Match with text from `kisim_diagnosis` based on RID, check again how much remains.
- Multiple diagnoses per patient are reasonable, but problem is automatically solved by excluding non-confirmed diagnoses as all of the RRMS diagnoses were made status_post which shouldn't be info in the text.
- Clean date column by using YYYY-MM-DD format, mapping YYYY to the YYYY-01-01 and YYYY-MM to YYYY-MM-01
- Exclude non-confirmed diagnoses like in old approach.

In [9]:
map_dict = {
    "Schubförmig remittierende Multiple Sklerose (RRMS)": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose": "relapsing_remitting_multiple_sclerosis",
    "Schubförmig remittierende Multiple Sklerose ": "relapsing_remitting_multiple_sclerosis",
    "Multiple Sklerose mit V.a. sekundär chronisch-progredienten Verlauf": "secondary_progressive_multiple_sclerosis",
    "Multiple Sklerose a.e. primär progredient": "primary_progressive_multiple_sclerosis",
    "Multiple Sklerose mit a.e. primär-progredientem Verlauf": "primary_progressive_multiple_sclerosis",
}

label_list = ["relapsing_remitting_multiple_sclerosis", "secondary_progressive_multiple_sclerosis", "primary_progressive_multiple_sclerosis"]

labels_cleaned = labels.replace(map_dict)
labels_cleaned = labels_cleaned[labels_cleaned["disease"].isin(label_list)]
print("Label distribution:")
print(labels_cleaned["disease"].value_counts(), "\n")
print("Unique patient IDs: ", len(labels_cleaned['research_id'].unique()))
print("Duplicate patient IDs: ", len(labels_cleaned['research_id']) - len(labels_cleaned['research_id'].unique()))

Label distribution:
disease
relapsing_remitting_multiple_sclerosis      286
secondary_progressive_multiple_sclerosis     28
primary_progressive_multiple_sclerosis       21
Name: count, dtype: int64 

Unique patient IDs:  328
Duplicate patient IDs:  7


In [10]:
# Date column to YYYY-MM-DD format with default 2023-01-01
labels_cleaned["diagnosis_date"] = labels_cleaned["diagnosis_date"].apply(lambda x: parser.parse(x, default=datetime.datetime(2023, 1, 1)).strftime("%Y-%m-%d"))

In [11]:
# Remove rows with non-confirmed diagnosis
labels_cleaned = labels_cleaned[labels_cleaned["diagnosis_reliability"] == "confirmed"]
# Keep only date, rid and disease and rename columns
labels_cleaned = labels_cleaned[["research_id", "diagnosis_date", "disease", "diagnosis_reliability"]].rename(columns={"research_id": "rid", "diagnosis_date": "date", "disease": "label"})

In [12]:
# Print the entries for the patients with multiple labels
print("Examples Patients with multiple labels:")
display((labels_cleaned[labels_cleaned["rid"].duplicated(keep=False)].sort_values(by=["rid", "date"])))

print("Summary of cleaned dataset: \n")
print(labels_cleaned.describe().iloc[:2], "\n")
print("shape: ", labels_cleaned.shape, "\n")
print("label distribution: \n", labels_cleaned["label"].value_counts(), "\n")
print("missing: \n", labels_cleaned.isna().sum(), "\n")

Examples Patients with multiple labels:


,rid,date,label,diagnosis_reliability


Summary of cleaned dataset: 

        rid date label diagnosis_reliability
count   260  260   260                   260
unique  260  149     3                     1 

shape:  (260, 4) 

label distribution: 
 label
relapsing_remitting_multiple_sclerosis      228
secondary_progressive_multiple_sclerosis     17
primary_progressive_multiple_sclerosis       15
Name: count, dtype: int64 

missing: 
 rid                      0
date                     0
label                    0
diagnosis_reliability    0
dtype: int64 



### After Cleaning

We end up with 228 cases of RRMS, 17 SPMS and 15 PPMS of 260 unique rids.

## Diagnosis text

The diagnosis text is in `kisim_diagnoses.csv`

In [13]:
text = pd.read_csv(paths.DATA_PATH_SEANTIS/'kisim_diagnoses.csv')

In [14]:
text.head()

,data_provider_code,diagnosis_code,diagnosis_code_system,diagnosis_code_system_version,diagnosis_date,diagnosis_encoded,diagnosis_id,diagnosis_label,diagnosis_ranking,diagnosis_role,LastUpdateDateTime,research_id
0,CHE-108.904.325,NaN,NaN,NaN,NaN,no,7024338|1,Kutane Karzinogenese \r\nSpinozelläre Karzinom...,primary,discharge,2021-01-19 08:34:58.0800000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
1,CHE-108.904.325,NaN,NaN,NaN,NaN,no,5255555|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2016-10-07 14:06:05.2630000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
2,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6435406|1,Aktuell kompensierte Harnblasensituation bei s...,primary,discharge,2020-04-28 06:27:53.2300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
3,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|5,"Schubförmige Multiple Sklerose\r\nES 03, ED 10...",secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1
4,CHE-108.904.325,NaN,NaN,NaN,NaN,no,6352273|4,Aktuell kompensierte Harnblasensituation bei s...,secondary,discharge,2019-07-24 11:21:01.4300000,AAE63006-727A-4DF6-82B7-32A86AB8F5C1


In [15]:
print("Description of text dataset: \n")
print(text.describe().iloc[:2], "\n")

print("Shape of text dataset: \n")
print(text.shape, "\n")

print("None values: \n")
print(text.isna().sum(), "\n")

Description of text dataset: 

       data_provider_code diagnosis_code diagnosis_code_system  \
count                7238           2101                  2101   
unique                  1            514                     1   

       diagnosis_code_system_version diagnosis_date diagnosis_encoded  \
count                           1808           2067              7238   
unique                             6            415                 2   

       diagnosis_id diagnosis_label diagnosis_ranking diagnosis_role  \
count          7238            7236              7238           7238   
unique         7238            3498                 2              1   

       LastUpdateDateTime research_id  
count                7238        7238  
unique               1505         275   

Shape of text dataset: 

(7238, 12) 

None values: 

data_provider_code                  0
diagnosis_code                   5137
diagnosis_code_system            5137
diagnosis_code_system_version    5430
diagno

In [16]:
# Unique patient IDs
print("Unique patient IDs: ", len(text['research_id'].unique()))

# Is there at least one date per rid?
print("There is at least one date per rid: ", text.groupby("research_id")["diagnosis_date"].apply(lambda x: x.notna().sum().all()))

Unique patient IDs:  275
There is at least one date per rid:  research_id
016B6D16-2BBA-4C05-A8E4-30F761C95813     True
08301EF0-1057-41AB-9911-772AE295E282    False
086D33C8-DDD3-4DE2-BFE0-A945076A2335    False
088B7604-2F11-459A-8C8C-8E1A19943333     True
08F1FA27-6213-47AF-B967-DC44CCCE9613     True
                                        ...  
F85F7220-F7BB-40EC-B2D9-A32EF15FFB86    False
F876F155-CC5E-4512-AED1-2991D22B5DE9    False
F87876FA-3068-4A93-AE71-6071CBE5F1F2     True
F99EA55D-F709-47AD-A9C4-69764B3FEFF0     True
FD303233-49CC-47B2-A5B4-5A50EA63327B     True
Name: diagnosis_date, Length: 275, dtype: bool


In [17]:
# Date column formats
print("Date column formats:")
print(text["diagnosis_date"].dropna().apply(lambda x: len(x)).value_counts())

Date column formats:
diagnosis_date
27    2067
Name: count, dtype: int64


In [18]:
print("Example date:")
print(text["diagnosis_date"].dropna().iloc[0])

Example date:
2018-12-06 18:53:00.0000000


In [19]:
# Get latest date per rid, then check if text is longest for that entry for that rid
text_lengths = text
text_lengths["text_length"] = text_lengths["diagnosis_label"].dropna().apply(lambda x: len(x))
longest_equal_newest = []
for rid in text_lengths["research_id"].unique():
    longest_text = text_lengths[text_lengths["research_id"] == rid].sort_values(by="text_length", ascending=False).iloc[0]["diagnosis_label"]
    newest_text = text_lengths[text_lengths["research_id"] == rid].sort_values(by="diagnosis_date", ascending=False).iloc[0]["diagnosis_label"]
    longest_equal_newest.append(longest_text == newest_text)

print("Longest text is equal to newest text: ", all(longest_equal_newest))
print(longest_equal_newest)

Longest text is equal to newest text:  False
[False, False, False, False, False, False, True, False, False, False, False, False, False, True, True, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, False, True, True, False, False, False, True, False, True, False, False, False, True, False, False, False, False, False, False, True,

## Problems
- There are multiple text entries per rid (diagnosis_label column). This is because doctors mostly just appended to existing reports, meaning the longest one should be the newest, or fullest report. 
- The LastUpdateDate is not useful for the dates, as different rows, that correspond to the same entries, share this date
- The diagnosis_date column has a lot of missing values. Because of missing values matching is hard to match using date.
- The date column is formated in the YYYY-MM-DD hh:mm:ss.ms format, could reformat

# Conclusions
- Multiple entries per rid, I will use the longest.
- I will clean the date column and fill the na values with the latest per rid (if it has even one). Date column might not be used for analysis but might prove useful for time-series follow-up.

In [20]:
# Relevant columns
text_cleaned = text[["research_id", "diagnosis_date", "diagnosis_label"]].rename(columns={"research_id": "rid", "diagnosis_date": "date", "diagnosis_label": "text"})

# Date column to YYYY-MM-DD format with default 2023-01-01 then set latest date per rid if date is missing
text_cleaned["date"] = text_cleaned["date"].apply(lambda x: parser.parse(str(x), default=datetime.datetime(2023, 1, 1)).strftime("%Y-%m-%d") if pd.notna(x) else x)
text_cleaned['date'] = pd.to_datetime(text_cleaned['date'])
text_cleaned['date'] = text_cleaned.groupby('rid')['date'].transform(lambda x: x.fillna(x.max()))

# Extract longest text per rid
text_cleaned = text_cleaned.groupby("rid").apply(lambda x: x.loc[x['text'].str.len().idxmax()]).reset_index(drop=True)

In [21]:
text_cleaned.head()

,rid,date,text
0,016B6D16-2BBA-4C05-A8E4-30F761C95813,2020-08-17,Diagnosen allgemein\r\n··Schubförmige Multiple...
1,08301EF0-1057-41AB-9911-772AE295E282,NaT,"Schubförmige Multiple Sklerose, ED 2006\r\n··E..."
2,086D33C8-DDD3-4DE2-BFE0-A945076A2335,NaT,"Schubförmige Multiple Sklerose (EM 12/2009, ED..."
3,088B7604-2F11-459A-8C8C-8E1A19943333,2015-10-30,"Schubförmig-remittierende Multiple Sklerose, E..."
4,08F1FA27-6213-47AF-B967-DC44CCCE9613,2016-03-09,"Schubförmig remittierende Multiple Sklerose, E..."


## Joining Datasets

We can join the datasets based on the rid. For rids with multiple entries we will use the date column to match them exactly. 

In [23]:
# Overlap of rid between labels and text
print("Unique patient IDs in labels: ", len(labels_cleaned['rid'].unique()))
print("Unique patient IDs in text: ", len(text_cleaned['rid'].unique()))
print("Overlap of rid between labels and text: ", len(set(labels_cleaned["rid"]).intersection(set(text_cleaned["rid"]))))

# Double labels
# Join labels and text on rid
double_labeled = labels_cleaned["rid"][labels_cleaned["rid"].duplicated(keep="first")]
print("Number of double labeled patients in both datasets: ", len(set(double_labeled).intersection(set(text_cleaned["rid"]))))

Unique patient IDs in labels:  260
Unique patient IDs in text:  275
Overlap of rid between labels and text:  199
Number of double labeled patients in both datasets:  0


In [24]:
# Join labels and text on rid
df = pd.merge(labels_cleaned, text_cleaned, on=["rid"], how="inner").drop(columns=["date_y", "diagnosis_reliability"]).rename(columns={"date_x": "date"})

# Set dtype of all columns to string
df = df.astype(str)

In [25]:
# Summary of cleaned dataset
print("Summary of cleaned dataset: \n")
print(df.info(), "\n")
print("shape: ", df.shape, "\n")
print("label distribution: \n", df["label"].value_counts(), "\n")
print("missing: \n", df.isna().sum(), "\n")

Summary of cleaned dataset: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rid     199 non-null    object
 1   date    199 non-null    object
 2   label   199 non-null    object
 3   text    199 non-null    object
dtypes: object(4)
memory usage: 6.3+ KB
None 

shape:  (199, 4) 

label distribution: 
 label
relapsing_remitting_multiple_sclerosis      173
secondary_progressive_multiple_sclerosis     13
primary_progressive_multiple_sclerosis       13
Name: count, dtype: int64 

missing: 
 rid      0
date     0
label    0
text     0
dtype: int64 



In [27]:
# Train test split
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train, val = train_test_split(train, test_size=0.1, random_state=42, stratify=train["label"])

# Save to csv
train.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_train.csv', index=False)
val.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_val.csv', index=False)
test.to_csv(paths.DATA_PATH_PREPROCESSED/'ms-diag/ms-diag_clean_test.csv', index=False)

In [ ]:
print("Train label distribution: \n", train["label"].value_counts(), "\n")
print("Val label distribution: \n", val["label"].value_counts(), "\n")
print("Test label distribution: \n", test["label"].value_counts(), "\n")

Train label distribution: 
 label
relapsing_remitting_multiple_sclerosis      124
secondary_progressive_multiple_sclerosis     10
primary_progressive_multiple_sclerosis        9
Name: count, dtype: int64 

Val label distribution: 
 label
relapsing_remitting_multiple_sclerosis      14
primary_progressive_multiple_sclerosis       1
secondary_progressive_multiple_sclerosis     1
Name: count, dtype: int64 

Test label distribution: 
 label
relapsing_remitting_multiple_sclerosis      35
primary_progressive_multiple_sclerosis       3
secondary_progressive_multiple_sclerosis     2
Name: count, dtype: int64 

